# Experimentation results

## Using old filtering system with 10 augmented and 1 real images processed through GPT Vision

### Results from researchers with next train transform:

In [94]:
# train_transform = transforms.Compose([
#             transforms.Resize(image_size),
#             transforms.RandomHorizontalFlip(p=0.5),
#             transforms.RandomRotation(degrees=15.0),
#             transforms.ToTensor(),
#             transforms.ConvertImageDtype(torch.float),
#             transforms.Lambda(lambda x: x.expand(3, *image_size)),
#             transforms.Normalize(mean=[0.5, 0.5, 0.5], 
#                                   std=[0.5, 0.5, 0.5])
#         ])

In [1]:
import pandas as pd
their_df = pd.read_csv('my_results/results_0_1.csv')
filtered_df = their_df[(their_df['metric'] == 'Accuracy') & 
                          (their_df['split'] == 'Validation')]

# Group by examples_per_class and get the max value for each group
best_results_their_v1 = filtered_df.groupby('examples_per_class')['value'].max().reset_index()
best_results_their_v1

FileNotFoundError: [Errno 2] No such file or directory: 'my_results/results_0_1.csv'

### Results from researchers with next train transform:

In [97]:
# transforms.Compose([
#             transforms.Resize(image_size),
#             transforms.ToTensor(),
#             transforms.ConvertImageDtype(torch.float),
#             transforms.Lambda(lambda x: x.expand(3, *image_size)),
#             transforms.Normalize(mean=[0.5, 0.5, 0.5], 
#                                   std=[0.5, 0.5, 0.5])
#         ])

In [113]:
import pandas as pd
their_df = pd.read_csv('my_results/results_0_1_v2.csv')
filtered_df = their_df[(their_df['metric'] == 'Accuracy') & 
                          (their_df['split'] == 'Validation')]

# Group by examples_per_class and get the max value for each group
best_results_their_v2 = filtered_df.groupby('examples_per_class')['value'].max().reset_index()
best_results_their_v2

,examples_per_class,value
0,1,0.649085


### My implementation results based on their code with next train transform:

In [106]:
# train_transform = transforms.Compose([
#             transforms.Resize(image_size),
#             transforms.RandomHorizontalFlip(p=0.5),
#             transforms.RandomRotation(degrees=15.0),
#             transforms.ToTensor(),
#             transforms.ConvertImageDtype(torch.float),
#             transforms.Lambda(lambda x: x.expand(3, *image_size)),
#             transforms.Normalize(mean=[0.5, 0.5, 0.5], 
#                                   std=[0.5, 0.5, 0.5])
#         ])

In [134]:
my_df = pd.read_csv('results_1_pascal-0-1_5.csv', index_col=None)
my_filtered_df = my_df[(my_df['metric'] == 'Accuracy') & 
                          (my_df['split'] == 'Validation')]

best_results_my_v1 = my_filtered_df.groupby('examples_per_class')['value'].max().reset_index()
best_results_my_v1

,examples_per_class,value
0,1,0.661101


In [121]:
best_results_my_v1.value.to_numpy()

array([0.66110075])

In [126]:
print(f"Difference between our and researchers implementation with augmentation transform: {best_results_their_v1.value.to_numpy()[0]-best_results_my_v1.value.to_numpy()[0]}")


Difference between our and researchers implementation with augmentation transform: 0.0018177000000000332


### My implementation results based on their code with next train transform:

In [108]:
# transforms.Compose([
#                 transforms.Resize(self.image_size),
#                 transforms.ToTensor(),
#                 transforms.ConvertImageDtype(torch.float),
#                 transforms.Lambda(lambda x: x.expand(3, *self.image_size)),
#                 transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
#             ])

In [124]:
my_df = pd.read_csv('results_1_pascal-0-1_4.csv', index_col=None)
my_filtered_df = my_df[(my_df['metric'] == 'Accuracy') & 
                          (my_df['split'] == 'Validation')]

best_results_my_v2 = my_filtered_df.groupby('examples_per_class')['value'].max().reset_index()
best_results_my_v2

,examples_per_class,value
0,1,0.659004


In [127]:
print(f"Difference between our and researchers implementation with simple transform: {best_results_their_v2.value.to_numpy()[0]-best_results_my_v2.value.to_numpy()[0]}")


Difference between our and researchers implementation with simple transform: -0.009919289999999914


In [136]:
my_df = pd.read_csv('results_1_pascal-0-1_7.csv', index_col=None)
my_filtered_df = my_df[(my_df['metric'] == 'Accuracy') & 
                          (my_df['split'] == 'Validation')]

best_results_my_v2 = my_filtered_df.groupby('examples_per_class')['value'].max().reset_index()
best_results_my_v2

,examples_per_class,value
0,1,0.661101


In [139]:
my_df = pd.read_csv('results_1_pascal-0-1_9.csv', index_col=None)
my_filtered_df = my_df[(my_df['metric'] == 'Accuracy') & 
                          (my_df['split'] == 'Validation')]

best_results_my_v2 = my_filtered_df.groupby('examples_per_class')['value'].max().reset_index()
best_results_my_v2

,examples_per_class,value
0,1,0.656475


In [143]:
my_df = pd.read_csv('results_1_pascal-0-1_10.csv', index_col=None)
my_filtered_df = my_df[(my_df['metric'] == 'Accuracy') & 
                          (my_df['split'] == 'Validation')]

best_results_my_v2 = my_filtered_df.groupby('examples_per_class')['value'].max().reset_index()
best_results_my_v2

,examples_per_class,value
0,1,0.656475


In [3]:
import pandas as pd
my_df = pd.read_csv('my_results/results_1_pascal-0-1_11.csv', index_col=None)
my_filtered_df = my_df[(my_df['metric'] == 'Accuracy') & 
                          (my_df['split'] == 'Validation')]

best_results_my_v2 = my_filtered_df.groupby('examples_per_class')['value'].max().reset_index()
best_results_my_v2

,examples_per_class,value
0,1,0.656475


In [68]:
my_filtered_df = my_df[(my_df['metric'] == 'Accuracy') & 
                          (my_df['split'] == 'Validation')]
my_filtered_df

,seed,examples_per_class,epoch,value,metric,split
3,0,1,0,0.50,Accuracy,Validation
87,0,1,1,0.50,Accuracy,Validation
171,0,1,2,0.55,Accuracy,Validation
255,0,1,3,0.65,Accuracy,Validation
339,0,1,4,0.65,Accuracy,Validation
423,0,1,5,0.65,Accuracy,Validation
507,0,1,6,0.65,Accuracy,Validation
591,0,1,7,0.65,Accuracy,Validation
675,0,1,8,0.65,Accuracy,Validation
759,0,1,9,0.65,Accuracy,Validation


,seed,examples_per_class,epoch,value,metric,split
0,0,1,0,1.978202,Loss,Training
1,0,1,0,2.326135,Loss,Validation
2,0,1,0,0.850066,Accuracy,Training
3,0,1,0,0.500000,Accuracy,Validation
4,0,1,0,1.988846,Loss Airplane,Training
...,...,...,...,...,...,...
4195,0,1,49,1.000000,Accuracy Train,Validation
4196,0,1,49,0.000453,Loss Television,Training
4197,0,1,49,0.364092,Loss Television,Validation
4198,0,1,49,1.000000,Accuracy Television,Training


In [70]:
their_df = pd.read_csv('my_results/results_0_1.csv')
filtered_df = their_df[(their_df['metric'] == 'Accuracy') & 
                          (their_df['split'] == 'Validation')]

best_results_their = filtered_df.groupby('examples_per_class')['value'].max().reset_index()
best_results_their

,examples_per_class,value
0,1,0.662918


In [81]:
df = pd.read_csv('results_1_pascal-0-1.csv')
filtered_df_my = df[(df['metric'] == 'Accuracy') & 
                          (df['split'] == 'Validation')]
# Group by examples_per_class and get the max value for each group
best_results_my = filtered_df_my.groupby('examples_per_class')['value'].max().reset_index()
best_results_my

,examples_per_class,value
0,1,0.64643


In [82]:
df = pd.read_csv('results_1_pascal-0-1_2.csv')
filtered_df_my = df[(df['metric'] == 'Accuracy') & 
                          (df['split'] == 'Validation')]
# Group by examples_per_class and get the max value for each group
best_results_my = filtered_df_my.groupby('examples_per_class')['value'].max().reset_index()
best_results_my

,examples_per_class,value
0,1,0.659004


In [84]:
their_df = pd.read_csv('my_results/results_0_1.csv')
filtered_df = their_df[(their_df['metric'] == 'Accuracy') & 
                          (their_df['split'] == 'Validation')]

best_results_their = filtered_df.groupby('examples_per_class')['value'].max().reset_index()
best_results_their

,examples_per_class,value
0,1,0.662918


## with simple transform

In [83]:
df = pd.read_csv('results_1_pascal-0-1_3.csv')
filtered_df_my = df[(df['metric'] == 'Accuracy') & 
                          (df['split'] == 'Validation')]
# Group by examples_per_class and get the max value for each group
best_results_my = filtered_df_my.groupby('examples_per_class')['value'].max().reset_index()
best_results_my

,examples_per_class,value
0,1,0.659004


In [89]:
0.662918 - 0.659004 

0.003913999999999973

In [92]:
0.662918 - 0.661101

0.0018169999999999575

In [85]:
df = pd.read_csv('results_1_pascal-0-1_4.csv')
filtered_df_my = df[(df['metric'] == 'Accuracy') & 
                          (df['split'] == 'Validation')]
# Group by examples_per_class and get the max value for each group
best_results_my = filtered_df_my.groupby('examples_per_class')['value'].max().reset_index()
best_results_my

,examples_per_class,value
0,1,0.659004


## With flips and rotations transform

In [87]:
df = pd.read_csv('results_1_pascal-0-1_5.csv')
filtered_df_my = df[(df['metric'] == 'Accuracy') & 
                          (df['split'] == 'Validation')]
# Group by examples_per_class and get the max value for each group
best_results_my = filtered_df_my.groupby('examples_per_class')['value'].max().reset_index()
best_results_my

,examples_per_class,value
0,1,0.661101


In [91]:
their_df = pd.read_csv('results_0_1.csv')
filtered_df = their_df[(their_df['metric'] == 'Accuracy') & 
                          (their_df['split'] == 'Validation')]

best_results_their = filtered_df.groupby('examples_per_class')['value'].max().reset_index()
best_results_their

,examples_per_class,value
0,1,0.649085


In [90]:
0.662918 - 0.661101

0.0018169999999999575

In [50]:
df = pd.read_csv('results_1_pascal-0-1_aws.csv')
df

,seed,examples_per_class,epoch,value,metric,split
0,0,1,0,1.978202,Loss,Training
1,0,1,0,2.326135,Loss,Validation
2,0,1,0,0.850066,Accuracy,Training
3,0,1,0,0.500000,Accuracy,Validation
4,0,1,0,1.988846,Loss Airplane,Training
...,...,...,...,...,...,...
4195,0,1,49,1.000000,Accuracy Train,Validation
4196,0,1,49,0.000453,Loss Television,Training
4197,0,1,49,0.364092,Loss Television,Validation
4198,0,1,49,1.000000,Accuracy Television,Training


In [53]:
their_df = pd.read_csv('my_results/results_0_1.csv')
their_df

,Unnamed: 0,seed,examples_per_class,epoch,value,metric,split
0,0,0,1,0,2.044756,Loss,Training
1,1,0,1,0,2.356342,Loss,Validation
2,2,0,1,0,0.827288,Accuracy,Training
3,3,0,1,0,0.536613,Accuracy,Validation
4,4,0,1,0,1.903856,Loss Airplane,Training
...,...,...,...,...,...,...,...
4195,4195,0,1,49,0.852861,Accuracy Train,Validation
4196,4196,0,1,49,0.000490,Loss Television,Training
4197,4197,0,1,49,0.491666,Loss Television,Validation
4198,4198,0,1,49,1.000000,Accuracy Television,Training


In [24]:
their_df = pd.read_csv('results_0_2.csv')

filtered_df = their_df[(their_df['metric'] == 'Accuracy') & 
                          (their_df['split'] == 'Validation')]

# Group by examples_per_class and get the max value for each group
best_results_their = filtered_df.groupby('examples_per_class')['value'].max().reset_index()
best_results_their

,examples_per_class,value
0,1,0.662918
1,2,0.711716


### Results from my implementation with top_k=10 detail=low

In [26]:
df = pd.read_csv('results_1_pascal-0-1.csv')
filtered_df = df[(df['metric'] == 'Accuracy') & 
                          (df['split'] == 'Validation')]

# Group by examples_per_class and get the max value for each group
best_results_my = filtered_df.groupby('examples_per_class')['value'].max().reset_index()
best_results_my

,examples_per_class,value
0,1,0.65


In [32]:
df = pd.read_csv('results_1_filtered-pascal-0-1-percentile-0_10.csv')
filtered_df = df[(df['metric'] == 'Accuracy') & 
                          (df['split'] == 'Validation')]

# Group by examples_per_class and get the max value for each group
best_results_my = filtered_df.groupby('examples_per_class')['value'].max().reset_index()
best_results_my

,examples_per_class,value
0,1,0.7


,seed,examples_per_class,epoch,value,metric,split
3,0,1,0,0.45,Accuracy,Validation
87,0,1,1,0.45,Accuracy,Validation
171,0,1,2,0.55,Accuracy,Validation
255,0,1,3,0.60,Accuracy,Validation
339,0,1,4,0.60,Accuracy,Validation
423,0,1,5,0.60,Accuracy,Validation
507,0,1,6,0.60,Accuracy,Validation
591,0,1,7,0.60,Accuracy,Validation
675,0,1,8,0.65,Accuracy,Validation
759,0,1,9,0.65,Accuracy,Validation


In [30]:
df = pd.read_csv('results_1_pascal-0-1_collab.csv')
filtered_df = df[(df['metric'] == 'Accuracy') & 
                          (df['split'] == 'Validation')]

# Group by examples_per_class and get the max value for each group
best_results_my = filtered_df.groupby('examples_per_class')['value'].max().reset_index()
best_results_my

,examples_per_class,value
0,1,0.7


In [33]:
df = pd.read_csv('results_1_filtered-pascal-0-1-percentile-0_10_collab.csv')
filtered_df = df[(df['metric'] == 'Accuracy') & 
                          (df['split'] == 'Validation')]

# Group by examples_per_class and get the max value for each group
best_results_my = filtered_df.groupby('examples_per_class')['value'].max().reset_index()
best_results_my

,examples_per_class,value
0,1,0.7


### Results from my implementation with top_k=9 detail=low

In [16]:
df = pd.read_csv('my_results/results_1_percentile_0_1.csv')
filtered_df = df[(df['metric'] == 'Accuracy') & 
                          (df['split'] == 'Validation')]

# Group by examples_per_class and get the max value for each group
best_results_my = filtered_df.groupby('examples_per_class')['value'].max().reset_index()
best_results_my

,examples_per_class,value
0,1,0.75


In [17]:
filtered_df

,seed,examples_per_class,epoch,value,metric,split
3,0,1,0,0.60,Accuracy,Validation
87,0,1,1,0.75,Accuracy,Validation
171,0,1,2,0.75,Accuracy,Validation
255,0,1,3,0.75,Accuracy,Validation
339,0,1,4,0.75,Accuracy,Validation
423,0,1,5,0.75,Accuracy,Validation
507,0,1,6,0.75,Accuracy,Validation
591,0,1,7,0.75,Accuracy,Validation
675,0,1,8,0.75,Accuracy,Validation
759,0,1,9,0.75,Accuracy,Validation


### Results from my implementation with percentile=0.1 detail=low

,examples_per_class,value
0,1,0.660436


### Results from my implementation with top_k=10 detail=auto

,examples_per_class,value
0,1,0.236185


### Results from my implementation with top_k=9 detail=auto

### Results from my implementation with percentile=0.1 detail=auto

## Other

In [13]:
import pandas as pd

# Load your dataset from the CSV file
# Update this to the correct file path
df = pd.read_csv('updated_augmented_images_with_class.csv')

# Calculate mean, median, and 75th percentile of overall_score
mean_score = df['overall_score'].mean()
median_score = df['overall_score'].median()
percentile_75_score = df['overall_score'].quantile(0.75)
percentile_25_score = df['overall_score'].quantile(0.25)

# Filter the data based on the calculated thresholds
mean_threshold_images = df[df['overall_score'] >= mean_score]
median_threshold_images = df[df['overall_score'] >= median_score]
percentile_75_threshold_images = df[df['overall_score'] >= percentile_75_score]
percentile_25_threshold_images = df[df['overall_score'] >= percentile_25_score]

# Count total images and images per class for each threshold
mean_threshold_counts = mean_threshold_images['class'].value_counts()
median_threshold_counts = median_threshold_images['class'].value_counts()
percentile_75_threshold_counts = percentile_75_threshold_images['class'].value_counts()
percentile_25_threshold_counts = percentile_25_threshold_images['class'].value_counts()

# Print the results
print("Mean Threshold:")
print(f"Total images above mean: {mean_threshold_images.shape[0]}")
print(f"Images per class above mean:\n{mean_threshold_counts}")

print("\nMedian Threshold:")
print(f"Total images above median: {median_threshold_images.shape[0]}")
print(f"Images per class above median:\n{median_threshold_counts}")

print("\n75th Percentile Threshold:")
print(f"Total images above 75th percentile: {percentile_75_threshold_images.shape[0]}")
print(f"Images per class above 75th percentile:\n{percentile_75_threshold_counts}")

print("\n25th Percentile Threshold:")
print(f"Total images above 75th percentile: {percentile_25_threshold_images.shape[0]}")
print(f"Images per class above 75th percentile:\n{percentile_25_threshold_counts}")


Mean Threshold:
Total images above mean: 90
Images per class above mean:
class
cow             10
boat            10
dog             10
potted plant    10
motorcycle       9
car              8
bicycle          6
bus              5
bird             4
sheep            4
sofa             4
chair            3
dining table     3
bottle           3
cat              1
Name: count, dtype: int64

Median Threshold:
Total images above median: 102
Images per class above median:
class
cow             10
boat            10
motorcycle      10
dog             10
potted plant    10
bicycle         10
car              8
bus              6
sheep            5
bird             4
sofa             4
airplane         4
chair            3
dining table     3
bottle           3
cat              2
Name: count, dtype: int64

75th Percentile Threshold:
Total images above 75th percentile: 53
Images per class above 75th percentile:
class
boat            10
car              8
motorcycle       8
potted plant     7
cow 